# Clasificación de Calidad de Datos Sensoriales — Modelo Binario

Este notebook implementa un modelo de Machine Learning para clasificar los
datos sensoriales como:

- **0 → Buenos (usables para modelado predictivo)**
- **1 → Malos (presentan algún tipo de anomalía detectada por el pipeline)**

El objetivo es evaluar cómo la limpieza y el preprocesamiento influyen en la
capacidad de un modelo ML para diferenciar datos confiables de datos
problemáticos.

Para ello:
- Se extraen mediciones desde la API (tabla `iot.mediciones`)
- Se genera un dataset con variables explicativas (`X`)
- Se usa como etiqueta la versión binaria de `indicador_calidad` (`y`)
- Se entrenan dos modelos:
  * Regresión Logística
  * Random Forest
- Se evalúa su desempeño con métricas estándar (accuracy, precision, recall, F1)
- Se construye la matriz de confusión y la curva ROC–AUC
- Se comparan resultados entrenando con datos sucios vs datos limpios


In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_ma trix, roc_auc_score, RocCurveDisplay
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pickle

API = "http://127.0.0.1:8000/api/mediciones"
HEADERS = {"x-api-key": "zxcvbnm"}

def load_mediciones(limit=50000):
    # puedes ajustar limit, pero ojo con tiempos de carga
    url = f"{API}?limit={limit}"
    r = requests.get(url, headers=HEADERS)
    r.raise_for_status()
    data = r.json()
    return pd.DataFrame(data)

df = load_mediciones(1000)
df.head()


""


In [5]:
df["quality_bin"] = np.where(df["indicador_calidad"] > 0, 1, 0)

KeyError: 'indicador_calidad'

In [ ]:
df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
df = df.dropna(subset=["valor"])

In [ ]:
df["variable_cat"] = df["variable"].astype("category").cat.codes

In [ ]:
X = df[["valor", "variable_cat"]].copy()
y = df["quality_bin"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)

In [ ]:
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42
)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

In [ ]:
def evaluate_model(name, y_true, y_pred):
    print(f"\n=== {name} ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1-score :", f1_score(y_true, y_pred))
    print("\nMatriz de confusión\n", confusion_matrix(y_true, y_pred))

# Evaluar LR
evaluate_model("Logistic Regression", y_test, y_pred_lr)

# Evaluar RF
evaluate_model("Random Forest", y_test, y_pred_rf)

In [ ]:
from sklearn.metrics import roc_curve, auc

rf_probs = rf.predict_proba(X_test)[:,1]
lr_probs = lr.predict_proba(X_test_scaled)[:,1]

print("AUC RF:", roc_auc_score(y_test, rf_probs))
print("AUC LR:", roc_auc_score(y_test, lr_probs))

fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_probs)

plt.figure(figsize=(6,6))
plt.plot(fpr_rf, tpr_rf, label="RF")
plt.plot(fpr_lr, tpr_lr, label="LR")
plt.plot([0,1], [0,1], "k--")
plt.legend()
plt.title("Curva ROC")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

Comparación entre “sucio” y “limpio”

In [ ]:
df_limpio = df[df["quality_bin"] == 0].copy()

Xl = df_limpio[["valor", "variable_cat"]]
yl = df_limpio["quality_bin"]  # siempre será 0 pero sirve para comparar

In [ ]:
rf_limpio = RandomForestClassifier(n_estimators=200, random_state=42)
rf_limpio.fit(Xl, yl)

pred_limpio = rf_limpio.predict(X_test)
evaluate_model("RF entrenado solo con datos limpios", y_test, pred_limpio)

In [ ]:
with open("modelo_calidad_rf.pkl", "wb") as f:
    pickle.dump(rf, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [ ]:
def clasificar_nuevo(valor, variable_cat):
    Xn = pd.DataFrame([[valor, variable_cat]], columns=["valor", "variable_cat"])
    pred = rf.predict(Xn)[0]
    return "BUENO" if pred == 0 else "MALO"

clasificar_nuevo(0.85, 3)